OCR整個資料夾 Chinese\img



In [6]:
from paddleocr import PaddleOCR, draw_ocr
import os
from PIL import Image

# 資料夾路徑設定
input_image_dir = r'C:\Users\zhuang\Desktop\Paddle\Chinese\img'
output_ocr_image_dir = r'C:\Users\zhuang\Desktop\Paddle\Chinese\ocrimg'
output_ocr_text_dir = r'C:\Users\zhuang\Desktop\Paddle\Chinese\ocrtxt'

# 檢查輸入資料夾是否存在
if not os.path.exists(input_image_dir):
    print(f"錯誤：輸入圖片資料夾 '{input_image_dir}' 不存在。請檢查路徑是否正確。")
    exit()

# 建立輸出資料夾
os.makedirs(output_ocr_image_dir, exist_ok=True)
os.makedirs(output_ocr_text_dir, exist_ok=True)

print(f"將從 '{input_image_dir}' 讀取圖片。")
print(f"OCR 可視化圖片將保存到： {output_ocr_image_dir}")
print(f"OCR 識別文字將保存到： {output_ocr_text_dir}")

# 初始化 OCR 模型（使用中文模型）
ocr = PaddleOCR(use_angle_cls=True, lang="ch", use_gpu=False)
font_path_for_drawing = r'C:\Windows\Fonts\msyh.ttc'

# 擷取圖片清單
image_files = [f for f in os.listdir(input_image_dir)
               if os.path.isfile(os.path.join(input_image_dir, f)) and
               f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]

if not image_files:
    print(f"在 '{input_image_dir}' 中沒有找到任何圖片文件。")
else:
    print(f"\n在 '{input_image_dir}' 中找到 {len(image_files)} 張圖片，開始進行 OCR 處理...")

    processed_count = 0

    for img_filename in sorted(image_files):
        full_image_path = os.path.join(input_image_dir, img_filename)
        print(f"\n正在處理圖片：'{img_filename}' (總計 {processed_count + 1}/{len(image_files)})")

        try:
            result = ocr.ocr(full_image_path, cls=True)

            recognized_texts = []
            if result and result[0]:
                for line in result[0]:
                    text = line[1][0]
                    recognized_texts.append(text)

                # 用原圖檔名儲存文字與圖像
                filename_no_ext = os.path.splitext(img_filename)[0]

                output_text_filename = f"{filename_no_ext}.txt"
                output_text_path = os.path.join(output_ocr_text_dir, output_text_filename)

                with open(output_text_path, 'w', encoding='utf-8') as f:
                    for text_line in recognized_texts:
                        f.write(text_line + '\n')
                print(f"  識別文字已保存到：{output_text_path}")

                try:
                    img = Image.open(full_image_path).convert('RGB')
                    boxes = [line[0] for line in result[0]]
                    txts = [line[1][0] for line in result[0]]
                    scores = [line[1][1] for line in result[0]]

                    im_show = draw_ocr(img, boxes, txts, scores, font_path=font_path_for_drawing)
                    im_show = Image.fromarray(im_show)

                    output_image_filename = f"{filename_no_ext}.jpg"
                    output_image_path = os.path.join(output_ocr_image_dir, output_image_filename)
                    im_show.save(output_image_path)
                    print(f"  可視化圖片已保存到：{output_image_path}")

                except ImportError:
                    print("\n警告：無法繪製識別結果。請安裝 'opencv-python' 和 'Pillow' 庫 (pip install opencv-python Pillow)。")
                except FileNotFoundError:
                    print(f"\n錯誤：找不到字體文件：'{font_path_for_drawing}'")
                except Exception as e:
                    print(f"\n繪製結果時發生未知錯誤: {e}")
            else:
                print(f"  未在圖片 '{img_filename}' 中識別到任何文字。")

            processed_count += 1

        except Exception as e:
            print(f"處理圖片 '{img_filename}' 時發生錯誤: {e}")

    print("\n所有圖片的 OCR 處理完成！")


將從 'C:\Users\zhuang\Desktop\Paddle\Chinese\img' 讀取圖片。
OCR 可視化圖片將保存到： C:\Users\zhuang\Desktop\Paddle\Chinese\ocrimg
OCR 識別文字將保存到： C:\Users\zhuang\Desktop\Paddle\Chinese\ocrtxt
[2025/06/09 23:01:05] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\zhuang/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\zhuang/.paddleocr/whl\\det\\ch\\ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_a

CER

In [10]:
import os
import re
import Levenshtein

# Ground Truth 和 預測結果資料夾路徑
gt_dir = r"C:\Users\zhuang\Desktop\Paddle\Chinese\txt"
pred_dir = r"C:\Users\zhuang\Desktop\Paddle\Chinese\ocrtxt"

# 清理文字：移除所有空白、換行、Tab 等
def clean_text(text):
    return re.sub(r'\s+', '', text)

# 統計變數
total_distance = 0
total_chars = 0
missing_pred_files = []

print("📄 每個檔案的 CER 結果：\n")

# 逐一處理 Ground Truth 資料夾中的每個 .txt 檔案
for filename in sorted(os.listdir(gt_dir)):
    if not filename.endswith(".txt"):
        continue

    gt_path = os.path.join(gt_dir, filename)
    pred_path = os.path.join(pred_dir, filename)

    if not os.path.exists(pred_path):
        missing_pred_files.append(filename)
        continue

    with open(gt_path, "r", encoding="utf-8") as f_gt, open(pred_path, "r", encoding="utf-8") as f_pred:
        gt_text = clean_text(f_gt.read())
        pred_text = clean_text(f_pred.read())

        distance = Levenshtein.distance(pred_text, gt_text)
        char_count = len(gt_text)
        file_cer = distance / char_count if char_count > 0 else 0

        print(f"{filename}：CER = {file_cer:.4f}")

        total_distance += distance
        total_chars += char_count

# 顯示總 CER
if total_chars > 0:
    total_cer = total_distance / total_chars
    print(f"\n✅ 總 CER：{total_cer:.4f}")
else:
    print("⚠️ 無有效字元可用於計算 CER。")

# 列出缺失的預測檔案
if missing_pred_files:
    print("\n⚠️ 找不到下列預測檔案：")
    for f in missing_pred_files:
        print(f" - {f}")


📄 每個檔案的 CER 結果：

1.txt：CER = 0.1006
10.txt：CER = 0.0926
2.txt：CER = 0.0602
3.txt：CER = 0.0604
4.txt：CER = 0.0283
5.txt：CER = 0.1003
6.txt：CER = 0.1013
7.txt：CER = 0.0297
8.txt：CER = 0.2000
9.txt：CER = 0.0367

✅ 總 CER：0.0698
